In [ ]:
import pandas as pd
import dataset
import sqlalchemy

In [ ]:
secop_2_items = pd.read_pickle('../data/secop_2_covid_items.pkl')

In [ ]:
secop_2_items.head()

In [ ]:
secop_covid = pd.read_pickle('../data/secop_union_all.pickle')

In [ ]:
secop_covid.columns

In [ ]:
secop_covid['valor_del_contrato'] = pd.to_numeric(secop_covid.valor_del_contrato)

In [ ]:
(secop_covid
     .loc[lambda x: x.is_covid]
     .groupby('departamento')
     .agg(valor_promedio=('valor_del_contrato', 'sum'))
     .reset_index()
     .valor_promedio.mean()
)

In [ ]:
(secop_covid
     .loc[lambda x: x.is_covid]
     .groupby('ciudad')
     .agg(valor_promedio=('valor_del_contrato', 'sum'))
     .reset_index()
     .valor_promedio.median()
)

In [ ]:
secop_2_covid = secop_covid.loc[lambda x: (x.is_covid) & (x.source == 'secop_2')].copy()
secop_2_covid.to_pickle('../data/secop_2_covid.pickle')

In [ ]:
secop_2_covid.shape

In [ ]:
- Codigo del contrato
- Item (codigo)
- Item (nombre)
- Item (cantidad)
- Nombre municipio
- Descripcion contrato
- Nombre contratista
- Valor del contrato
- Link secop

In [ ]:
secop_join = pd.merge(
    secop_2_covid,
    secop_2_items,
    how='left',
    on='id_contrato'
)

In [ ]:
secop_join.columns

In [ ]:
secop_join = secop_join[['nombre_entidad', 'nit_entidad', 'departamento', 'ciudad',
            'proceso_de_compra', 'id_contrato',
            'descripcion_del_proceso', 'tipo_de_contrato',
            'modalidad_de_contratacion',
            'documento_proveedor', 'proveedor_adjudicado',
            'url',
            'valor_del_contrato',
            'item_code', 'item_description', 'item_quantity',
            'item_unit', 'item_price']]

secop_join.to_csv('../data/secop_2_covid_items_join.csv')

## Alarm

In [ ]:
from fuzzywuzzy import fuzz, process

In [ ]:
precios = pd.read_csv('../data/precios.csv')

In [ ]:
precios.head()

In [ ]:
precios = (precios
 .loc[lambda x: x.precio_techo!='-']
 .assign(precio_techo = lambda x: pd.to_numeric(x.precio_techo, errors='coerce'))
)

In [ ]:
precios = precios.reset_index().rename(columns={'index':'id'})

In [ ]:
# save this verison of precios for future reference
precios.to_feather('../data/precios_reference.feather')
precios.to_pickle('../data/precios_reference.pickle')

In [ ]:
secop_join['items_per_contract'] = secop_join.groupby('id_contrato')['item_description'].transform('count')
secop_join_valid = secop_join.loc[lambda x: ~(x.items_per_contract == 1) | ~(x.item_quantity == 1)]

In [ ]:
secop_join_valid = secop_join_valid.loc[lambda x: x.tipo_de_contrato.isin(['Compraventa', 'Suministros'])]

In [ ]:
secop_join_valid.groupby('tipo_de_contrato').agg(n=('id_contrato', 'count'))

preprocess item texts and external items

In [ ]:
from toolz import curry
from toolz.functoolz import pipe


@curry
def remove_pattern(pattern, series: pd.Series):
    return series.str.replace(pattern, '')


class TextPreprocessor:

    def __init__(self):
        self._init_pipeline()

    def _init_pipeline(self):
        # Get rid of URLs
        text_operations = [
            # Get rid of URLs
            remove_pattern('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'),
            # Take out retweet header, there is only one
            remove_pattern('RT @[a-z,A-Z]*: '),
            # Get rid of hashtags
            remove_pattern('#'),
            # Get rid of references to other screennames
            remove_pattern('@[a-z,A-Z]*'),
            # everything thats no a number or space
            remove_pattern('[^\w\s]'),
            lambda series: series.str.replace('\s\s+', ' '),
            # strip series entries
            lambda series: series.str.strip()
        ]

        self.text_pipeline = lambda data: pipe(data, *text_operations)

    def preprocess(self, texts: pd.Series):
        return self.text_pipeline(texts)


In [ ]:
secop_join_valid['item_description'] = secop_join_valid.item_description.str.lower()
precios['producto'] = precios.producto.str.lower()

In [ ]:
preprocessor = TextPreprocessor()
secop_join_valid['item_description_clean'] = preprocessor.preprocess(secop_join_valid.item_description)
precios['producto_clean'] = preprocessor.preprocess(precios.producto)

In [ ]:
class ItemMatcher:
    def __init__(self, items, score_cutoff=80):
        # TODO separate into price and 
        self.items = items
        self.score_cutoff = score_cutoff
        
    def find_match(self, item_description):
        if item_description:
            return process.extractOne(item_description, self.items, scorer = fuzz.partial_ratio, score_cutoff=self.score_cutoff)
        else:
            return None

In [ ]:
item_matcher = ItemMatcher(precios.producto_clean, score_cutoff=0)

Only in suministros and compraventa

Matches

In [ ]:
item_matches = secop_join_valid.item_description_clean.apply(item_matcher.find_match)

In [ ]:
item_matches_df = pd.DataFrame(item_matches.tolist(), columns = ['closest_match', 'score', 'match_id'])
item_matches_df = item_matches_df.loc[lambda x: x.score >= 70]

In [ ]:
item_matches_df = (
    pd.merge(item_matches_df, precios, left_on='match_id', right_on = 'id')
    .drop(columns=['closest_match', 'id'])
)

In [ ]:
secop_join_valid_w_match = secop_join_valid.join(item_matches_df, how='left')

In [ ]:
secop_join_valid_w_match = secop_join_valid_w_match.assign(
    alarma_sobrecosto = lambda x: x.item_price > x.precio_techo
)

In [ ]:
secop_join_valid_w_match.to_csv('../data/secop_join_suministros_w_sobrecosto.csv')

Contracts with no or with a single item

In [ ]:
secop_join_singleitem = (secop_join
    .loc[lambda x: (x.item_code.isnull()) | ((x.items_per_contract == 1) & (x.item_quantity == 1))]
    .loc[lambda x: x.tipo_de_contrato.isin(['Compraventa', 'Suministros'])]
)

In [ ]:
(secop_join_singleitem
     .drop(['item_code',
       'item_description', 'item_quantity', 'item_unit', 'item_price'], axis = 1)
     .to_csv('../data/secop_2_covid_singleitem.csv')
)